In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize

from collections.abc import Mapping
import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS, IVGMMCUE

import os
import multiprocessing as mp

os.chdir("/home/joosungm/projects/def-lelliott/joosungm/projects/ssc23-case-comp")

In [3]:
prov_short = ["AB", "BC", "MB", "NB", "NL", "NS", "ON", "PE", "QC", "SK"]

scales = ["raw_", "log_"]

treatments = ["tmax", "tavg", "tdiff"]

In [4]:
treatment = "tavg"
scale = "log_"
i_prov = 1 # BC

prod_filename = "./data/user_data/01_iv_analysis/" + prov_short[i_prov] + "/prod_temp.csv"

prod_data = pd.read_csv(prod_filename)
prod_data = prod_data.rename(columns = {"Date":"date"})
prod_data["tdiff"] = np.abs(prod_data.tmax - prod_data.tmin)
prod_data["log_population"] = np.log(prod_data.Population + 1)
# prod_data.head()

# - load covariates
covar_df = pd.read_csv("./data/user_data/_covariates/ppi_and_usd_imputed.csv")
# covar_df.head()

# - min max scale each column of covar_df
from sklearn.preprocessing import MinMaxScaler
covar_sub = covar_df.drop(columns = ["date", "year", "month"])
covar_scaled = pd.DataFrame(MinMaxScaler().fit_transform(covar_sub), columns = covar_sub.columns)
# covar_sub.head()

# - apply PCA to covar_scaled
n_pca = 5
pca = PCA(n_components = n_pca)
covar_pca = pca.fit_transform(covar_scaled)
covar_pca = pd.DataFrame(covar_pca, columns = ["PC"+str(i) for i in range(1, n_pca+1)])

# - add date column to covar_pca
covar_df = pd.concat([covar_df.loc[:, "date"], covar_pca], axis = 1)
# covar_df.head()


# - left join the covariates to the prod_data by date.
full_df = pd.merge(prod_data, covar_df, on='date', how='left')

# ppi_covars = ["PC"+str(i) for i in range(1, n_pca+1)]
ppi_covars = covar_pca.columns.tolist()
# ppi_covars
# apply lags to ppi_covars
lags = 3
for lag in range(1, lags+1):
    for covar in ppi_covars:
        full_df[covar + "_lag_" + str(lag)] = full_df[covar].shift(lag)

# drop first nlags rows
full_df = full_df.dropna()
# full_df.head()

# Create example data for analysis
# - Subset date, lat, long, the covariates, tavg, and production_in_division_X31.33.Manufacturing from full_df
# and then rename the column to 'production'
prods = prod_data.columns[2:16]

i = 9
example_prod = prods[i]  # Agriculture, Forestry, Fishing and Hunting & oil and gas extraction
temperatures = ["tavg", "tmin", "tmax"]
instruments = ["lat", "long"]

eff_modifiers = [s + "_lag_"+str(lags) for s in ppi_covars]
# eff_modifiers.append("Population")
eff_modifiers.append("log_population")
com_causes = ["year", "month"]

example_cols = [example_prod] + eff_modifiers + com_causes + temperatures + instruments + ["date"]
example_df = full_df.loc[:, example_cols].reset_index(drop= True)
example_df = example_df.rename(columns = {example_prod:"production"})

# create log(production) column. First add 1 to all values of production to avoid taking log of 0.
example_df.loc[:, "log_production"] = np.log(example_df.production + 1)
example_df.loc[:, "tdiff"] = np.abs(example_df.tmax - example_df.tmin)


# plot histogram of log_production
# import matplotlib.pyplot as plt
# plt.hist(example_df.log_production)
# help(np.log)

# create columns for the seasons of the year
example_df.loc[:, "spring"] = (example_df.month.isin([3,4,5])).astype(int)
example_df.loc[:, "summer"] = (example_df.month.isin([6,7,8])).astype(int)
example_df.loc[:, "fall"] = (example_df.month.isin([9,10,11])).astype(int)
example_df.loc[:, "winter"] = (example_df.month.isin([12,1,2])).astype(int)

# example_df.head(5)
covariates = [s + "_lag_"+str(lags) for s in ppi_covars] + ["log_population", "year", "month", "spring", "summer", "fall"]
covariates 

['PC1_lag_3',
 'PC2_lag_3',
 'PC3_lag_3',
 'PC4_lag_3',
 'PC5_lag_3',
 'log_population',
 'year',
 'month',
 'spring',
 'summer',
 'fall']

In [5]:
# Perform IV analysis
# - Here we fit 4 models:
#   1. OLS
#   2. IV with lat as instrument
#   3. IV with long as instrument
#   4. IV with both lat and long as instruments

import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS, IVGMMCUE
from collections import OrderedDict
from linearmodels.iv.results import compare

iv_ols = IV2SLS(
    dependent = example_df["log_production"],
    exog = example_df.loc[:, covariates + ["tavg"]],
    endog = None,
    instruments = None
).fit()

iv_lat = IV2SLS(
    dependent = example_df["log_production"],
    exog = example_df.loc[:, covariates],
    endog = example_df.loc[:, treatment],
    instruments = example_df.loc[:, ["lat"]]
).fit()

iv_long = IV2SLS(
    dependent = example_df["log_production"],
    exog = example_df.loc[:, covariates],
    endog = example_df.loc[:, treatment],
    instruments = example_df.loc[:, ["long"]]
).fit()    

iv_both = IV2SLS(
    dependent = example_df["log_production"],
    exog = example_df.loc[:, covariates],
    endog = example_df.loc[:, treatment],
    instruments = example_df.loc[:, ["lat", "long"]],
).fit()

ivs = OrderedDict()
ivs["ols"] = iv_ols
ivs["lat"] = iv_lat
ivs["long"] = iv_long
ivs["both"] = iv_both

print(compare(ivs))
# The parentheses contain the T-stats.

/home/joosungm/venvs/py310/lib/python3.10/site-packages/linearmodels/iv/data.py:30: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  if is_categorical(s):
/home/joosungm/venvs/py310/lib/python3.10/site-packages/linearmodels/iv/data.py:30: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  if is_categorical(s):
/home/joosungm/venvs/py310/lib/python3.10/site-packages/linearmodels/iv/data.py:30: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  if is_categorical(s):
/home/joosungm/venvs/py310/lib/python3.10/site-packages/linearmodels/iv/data.py:30: FutureWarning: is_categorical is deprecated and will be removed in a future version. Use is_categorical_dtype instead.
  if is_categorical(s):


                                        Model Comparison                                        
                                    ols                lat               long               both
------------------------------------------------------------------------------------------------
Dep. Variable            log_production     log_production     log_production     log_production
Estimator                           OLS            IV-2SLS            IV-2SLS            IV-2SLS
No. Observations                 141114             141114             141114             141114
Cov. Est.                        robust             robust             robust             robust
R-squared                        0.9282             0.9182             0.7200             0.9133
Adj. R-squared                   0.9282             0.9182             0.7200             0.9133
F-statistic                   1.541e+06          1.492e+06          5.146e+05          1.441e+06
P-value (F-stat)              

/home/joosungm/venvs/py310/lib/python3.10/site-packages/linearmodels/iv/results.py:1474: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  vals = concat(
/home/joosungm/venvs/py310/lib/python3.10/site-packages/linearmodels/compat/pandas.py:43: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  return pd.concat(*args, **kwargs)
/home/joosungm/venvs/py310/lib/python3.10/site-packages/linearmodels/compat/pandas.py:43: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  return pd.concat(*args, **kwargs)


In [6]:
# Check the validity of the instruments

# - Testing for overidentification.
print(iv_both.wooldridge_overid)

# Overidentification exists. One of lat or long appears to be endogenous.
# - The similarity between the coefficient of tavg in OLS and iv_lat suggest that lat may be endogenous.
#   Therefore, we will use "long" as our instrumental variable for tavg.

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 1581.7086
P-value: 0.0000
Distributed: chi2(1)


In [62]:
# - Testing for endogeneity of tavg
# print(iv_lat.wu_hausman())
print(iv_long.wu_hausman())
# print(iv_both.wu_hausman())

# The result shows that tavg is endogenous.

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 1152.4718
P-value: 0.0000
Distributed: F(1,141101)


In [24]:
df = iv_lat.params.to_frame()
df

,parameter
PC1_lag_3,0.072915
PC2_lag_3,-0.006235
PC3_lag_3,-0.043300
PC4_lag_3,0.166764
PC5_lag_3,-0.033326
log_population,0.710084
year,-0.001420
month,-0.003581
spring,0.669740
summer,1.445329


In [25]:
df.loc["tavg", "parameter"]

-0.08024364617091706

In [30]:
iv_model = iv_both
print(iv_model.wooldridge_overid.pval)
print(iv_model.wu_hausman().pval)

0.0
1.1102230246251565e-16


In [34]:
print(iv_lat.params["tavg"])
print(iv_lat.pvalues["tavg"])

-0.08024364617091706
0.0


In [37]:
lat_diff = np.abs(iv_ols.params[treatment] - iv_lat.params[treatment])
long_diff = np.abs(iv_ols.params[treatment] - iv_long.params[treatment])
print(lat_diff)
print(long_diff)

0.05339072615431739
0.24389881580554196
